# IMDB Dataset - Capstone Project

# Part:1 - Web scraping

In [1]:
#importing libraries
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import warnings
warnings.filterwarnings('ignore')
import time
from selenium.webdriver.edge.service import Service
import requests
from bs4 import BeautifulSoup
import re
from itertools import groupby
import csv

In [2]:
# Setting the path to the Microsoft Edge WebDriver executable
msedgedriver_path = r"C:\Users\jdjra\Desktop\Internship\msedgedriver.exe"

# Creating an Edge WebDrivr instance using the specified path
edge_service = Service(msedgedriver_path)
driver = webdriver.Edge(service=edge_service)

# Navigating to the Url
driver.get('https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1')

# Maximizing the window
driver.maximize_window()

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [3]:
# Creating empty lists
Header = []
Details = []
Rat_bar = []
Names = []
Votes_gross = []

# Looping pages to scrape data
for page in range(37):
    # Getting movie titles and append to Header list
    title = driver.find_elements(By.XPATH, '//h3[@class="lister-item-header"]')
    for i in title:
        Header.append(i.text)

    # Getting movie details and append to Details list
    row2 = driver.find_elements(By.XPATH, '//p[@class="text-muted "]')
    for i in row2:
        Details.append(i.text)

    # Getting movie ratings bars and append to Rat_bar list
    rtng = driver.find_elements(By.XPATH, '//div[@class="ratings-bar"]')
    for i in rtng:
        Rat_bar.append(i.text)

    # Getting movie names and stars, and append to Names list
    syn = driver.find_elements(By.XPATH, '//p[contains(., "Stars") or contains(., "Star")]')
    for i in syn:
        star = i.find_elements(By.XPATH, './/a')
        for a in star:
            Names.append(i.text)

    # Getting movie votes/gross and append to Votes_gross list
    gross = driver.find_elements(By.XPATH, '//p[@class="sort-num_votes-visible"]')
    for i in gross:
        Votes_gross.append(i.text) 
    time.sleep(2)

    # Try to click the "Next" button for the next page
    try:
        next_button = driver.find_element(By.XPATH, '//a[contains(., "Next")]')
        next_button.click()
        time.sleep(1)
    except:
        # Break the loop if there is no "Next" button (reached the last page)
        break

#Checking the lenght of lists 
print(len(Header),len(Details),len(Rat_bar),len(Names),len(Votes_gross))

1775 1775 1775 9010 1775


In [4]:
#Checking the format of scraped data
Header

['1. The Dark Knight (2008)',
 '2. The Lord of the Rings: The Return of the King (2003)',
 '3. Spider-Man: Across the Spider-Verse (2023)',
 '4. Inception (2010)',
 '5. The Lord of the Rings: The Fellowship of the Ring (2001)',
 '6. The Lord of the Rings: The Two Towers (2002)',
 '7. The Matrix (1999)',
 '8. Star Wars: Episode V - The Empire Strikes Back (1980)',
 '9. Soorarai Pottru (2020)',
 '10. Terminator 2: Judgment Day (1991)',
 '11. Star Wars: Episode IV - A New Hope (1977)',
 '12. Seven Samurai (1954)',
 '13. Harakiri (1962)',
 '14. Sita Ramam (2022)',
 '15. Gladiator (2000)',
 '16. Leon (1994)',
 '17. The Dark Knight Rises (2012)',
 '18. Spider-Man: Into the Spider-Verse (2018)',
 '19. Raiders of the Lost Ark (1981)',
 '20. Avengers: Endgame (2019)',
 '21. Aliens (1986)',
 '22. Oldboy (2003)',
 '23. Avengers: Infinity War (2018)',
 '24. Kaithi (2019)',
 '25. Asuran (2019)',
 '26. Top Gun: Maverick (2022)',
 '27. Heat (1995)',
 '28. Star Wars: Episode VI - Return of the Jedi (1

In [6]:
# Extract the movie name from the Header list
Movie_name = [item.split(' (')[0].split('. ')[1] for item in Header]

# Extract the year of the movie release from the 'Header' list and convert to int
Year = [int(item.split('(')[-1].strip(')').replace(',', '')) for item in Header]

# Create a list of serial numbers from 1 to 1772
sr_number = [num for num in range(1, 1776)]

# Check the length of the created lists
print(len(Movie_name), len(Year), len(sr_number))

1775 1775 1775


In [7]:
#Checking the format of scraped data
Details

['12A | 152 min | Action, Crime, Drama',
 '12A | 201 min | Action, Adventure, Drama',
 'PG | 140 min | Animation, Action, Adventure',
 '12A | 148 min | Action, Adventure, Sci-Fi',
 'PG | 178 min | Action, Adventure, Drama',
 '12A | 179 min | Action, Adventure, Drama',
 '15 | 136 min | Action, Sci-Fi',
 'U | 124 min | Action, Adventure, Fantasy',
 '15 | 153 min | Action, Drama',
 '15 | 137 min | Action, Sci-Fi',
 'U | 121 min | Action, Adventure, Fantasy',
 'A | 207 min | Action, Drama',
 'X | 133 min | Action, Drama, Mystery',
 '12A | 163 min | Action, Drama, Mystery',
 '15 | 155 min | Action, Adventure, Drama',
 '18 | 110 min | Action, Crime, Drama',
 '12A | 164 min | Action, Drama, Thriller',
 'PG | 117 min | Animation, Action, Adventure',
 'A | 115 min | Action, Adventure',
 '12A | 181 min | Action, Adventure, Drama',
 'U | 137 min | Action, Adventure, Sci-Fi',
 '18 | 101 min | Action, Drama, Mystery',
 '12A | 149 min | Action, Adventure, Sci-Fi',
 '12A | 145 min | Action, Adventure

In [8]:
# Creating empty lists to store data
Certification = []
Duration = []
Genre = []

# Iterate each item in the 'Details' list and split it into parts based on '|'
# Extracting movie certification, duration, and genre, and append
# If 'Details' has less than 3 parts, then assume certification is not available ('N/A') because there are movies with no Certification
for item in Details:
    parts = item.split('|')
    if len(parts) < 3:
        Certification.append('N/A')
        Duration.append(parts[0].strip())
        Genre.append(parts[1].strip())
    else:
        Certification.append(parts[0].strip())
        Duration.append(parts[1].strip())
        Genre.append(parts[2].strip())

# Check the length of the created lists to ensure they have the same number of elements
print(len(Certification), len(Duration), len(Genre))

1775 1775 1775


In [9]:
#Checking the format of scraped data
Rat_bar

['9.0 Rate this\n84 Metascore',
 '9.0 Rate this\n94 Metascore',
 '8.9 Rate this\n86 Metascore',
 '8.8 Rate this\n74 Metascore',
 '8.8 Rate this\n92 Metascore',
 '8.8 Rate this\n87 Metascore',
 '8.7 Rate this\n73 Metascore',
 '8.7 Rate this\n82 Metascore',
 '8.7 Rate this',
 '8.6 Rate this\n75 Metascore',
 '8.6 Rate this\n90 Metascore',
 '8.6 Rate this\n98 Metascore',
 '8.6 Rate this\n85 Metascore',
 '8.6 Rate this',
 '8.5 Rate this\n67 Metascore',
 '8.5 Rate this\n64 Metascore',
 '8.4 Rate this\n78 Metascore',
 '8.4 Rate this\n87 Metascore',
 '8.4 Rate this\n85 Metascore',
 '8.4 Rate this\n78 Metascore',
 '8.4 Rate this\n84 Metascore',
 '8.4 Rate this\n77 Metascore',
 '8.4 Rate this\n68 Metascore',
 '8.4 Rate this',
 '8.4 Rate this',
 '8.3 Rate this\n78 Metascore',
 '8.3 Rate this\n76 Metascore',
 '8.3 Rate this\n58 Metascore',
 '8.3 Rate this\n76 Metascore',
 '8.3 Rate this\n98 Metascore',
 '8.3 Rate this',
 '8.3 Rate this',
 '8.3 Rate this',
 '8.3 Rate this',
 '8.3 Rate this',
 '8.3 

In [10]:
# Creating empty lists
Ratings = []
Metascore = []

# Iterate each item in the 'Rat_bar' list
# Use 'Regular Expressions' for extracting the movie rating and Metascore
# If ratings or Metascore are not found, set them as 'N/A'
for item in Rat_bar:
    rating_match = re.search(r'([\d.]+) Rate this', item)
    metascore_match = re.search(r'(\d+) Metascore', item)
    rating = rating_match.group(1) if rating_match else 'N/A'
    Ratings.append(float(rating))
    metascore = metascore_match.group(1) if metascore_match else 'N/A'
    Metascore.append(int(metascore) if metascore != 'N/A' else metascore)

# Check the length of the created lists
print(len(Ratings), len(Metascore))

1775 1775


In [11]:
#Checking the format of scraped data
Names

['Director: Christopher Nolan | Stars: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine',
 'Director: Christopher Nolan | Stars: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine',
 'Director: Christopher Nolan | Stars: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine',
 'Director: Christopher Nolan | Stars: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine',
 'Director: Christopher Nolan | Stars: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine',
 'Director: Peter Jackson | Stars: Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom',
 'Director: Peter Jackson | Stars: Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom',
 'Director: Peter Jackson | Stars: Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom',
 'Director: Peter Jackson | Stars: Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom',
 'Director: Peter Jackson | Stars: Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom',
 'Directors: Joaquim Dos 

* The scraped data have many repeating values therefore we need to remove duplicates. 
* After removing duplicates the len(Names) is lower than that of other lists of scraped data suggesting that some data was lost on directly removal of duplicates. 
* The reason found was that there are many sequel movies with same Director and Stars. 
* In order to tackle the problem we are going to use 'groupby' from itertools library.
* Which will remove only continous repeating values.

In [12]:
# Creating function to remove continuous duplicates from the 'Names' list using 'groupby'
def remove_continuous_duplicates(Names):
    return [k for k, _ in groupby(Names)]

#Creating new list Names1
Names1 = remove_continuous_duplicates(Names)

#Checking length of Names1
len(Names1)

1775

In [13]:
Names1

['Director: Christopher Nolan | Stars: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine',
 'Director: Peter Jackson | Stars: Elijah Wood, Viggo Mortensen, Ian McKellen, Orlando Bloom',
 'Directors: Joaquim Dos Santos, Kemp Powers, Justin K. Thompson | Stars: Shameik Moore, Hailee Steinfeld, Brian Tyree Henry, Luna Lauren Velez',
 'Director: Christopher Nolan | Stars: Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page, Ken Watanabe',
 'Director: Peter Jackson | Stars: Elijah Wood, Ian McKellen, Orlando Bloom, Sean Bean',
 'Director: Peter Jackson | Stars: Elijah Wood, Ian McKellen, Viggo Mortensen, Orlando Bloom',
 'Directors: Lana Wachowski, Lilly Wachowski | Stars: Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving',
 'Director: Irvin Kershner | Stars: Mark Hamill, Harrison Ford, Carrie Fisher, Billy Dee Williams',
 'Director: Sudha Kongara | Stars: Suriya, Paresh Rawal, Aparna Balamurali, Urvashi',
 'Director: James Cameron | Stars: Arnold Schwarzenegger, Li

Duplicate values are no longer present.

In [14]:
#Creating empty lists
Directors = []
Stars = []

# Iterate through each item in the 'Names1' list
# Split each item into parts based on '| Stars: ' or '| Star: ' (if applicable)
# Extract movie directors and stars from the parts and append them to their respective lists
for item in Names1:
    parts = item.split('| Stars: ')
    if len(parts) == 1:
        parts = item.split('| Star: ')

    # Extracting Directors
    if 'Directors:' in parts[0]:
        directors = parts[0].split('Directors: ')[1].strip()
    else:
        directors = parts[0].split('Director: ')[1].strip()

    # Extracting Stars
    stars = [star.strip() for star in parts[1].split(',')] if len(parts) > 1 else []
    Directors.append(directors)
    Stars.append(stars)
    
#Checking length of created lists
print(len(Directors),len(Stars))

1775 1775


In [15]:
#Checking the format of scraped data
Votes_gross

['Votes: 2,756,999 | Gross: $534.86M',
 'Votes: 1,905,424 | Gross: $377.85M',
 'Votes: 200,317',
 'Votes: 2,447,577 | Gross: $292.58M',
 'Votes: 1,933,646 | Gross: $315.54M',
 'Votes: 1,719,437 | Gross: $342.55M',
 'Votes: 1,977,968 | Gross: $171.48M',
 'Votes: 1,334,139 | Gross: $290.48M',
 'Votes: 120,244',
 'Votes: 1,134,882 | Gross: $204.84M',
 'Votes: 1,406,155 | Gross: $322.74M',
 'Votes: 356,006 | Gross: $0.27M',
 'Votes: 62,707',
 'Votes: 61,843',
 'Votes: 1,551,728 | Gross: $187.71M',
 'Votes: 1,201,633 | Gross: $19.50M',
 'Votes: 1,763,263 | Gross: $448.14M',
 'Votes: 616,626 | Gross: $190.24M',
 'Votes: 1,007,400 | Gross: $248.16M',
 'Votes: 1,202,299 | Gross: $858.37M',
 'Votes: 739,140 | Gross: $85.16M',
 'Votes: 604,124 | Gross: $0.71M',
 'Votes: 1,144,058 | Gross: $678.82M',
 'Votes: 36,984',
 'Votes: 31,651',
 'Votes: 618,475 | Gross: $718.73M',
 'Votes: 686,392 | Gross: $67.44M',
 'Votes: 1,087,496 | Gross: $309.13M',
 'Votes: 412,154 | Gross: $2.38M',
 'Votes: 337,196

In [16]:
Gross_col=[]
Votes=[]

# Iterate each item in the 'Votes_gross' list
# Split each item into parts by ' | Gross: '
# Extract votes and append
for i in Votes_gross:
    parts = i.split(' | Gross: ')
    Votes.append(parts[0].split('Votes: ')[1].strip())

    if len(parts) > 1:
        Gross_col.append(parts[1])
    else:
        Gross_col.append('N/A')

# Checknig the length of cerated lists
print(len(Gross_col), len(Votes))

1775 1775


Following data is scraped:
* sr_number
* Movie_name
* Year
* Certification
* Genre
* Duration
* Ratings
* Metascore
* Directors
* Stars
* Gross_col
* Votes

In [17]:
#Checking the lenght of all the scraped lists
print(len(sr_number),len(Movie_name),len(Year),len(Certification),len(Genre),len(Duration),len(Ratings),len(Metascore),len(Gross_col),len(Votes),len(Directors),len(Stars))

1775 1775 1775 1775 1775 1775 1775 1775 1775 1775 1775 1775


In [18]:
Duration_in_min=[int(i.split(' ')[0]) for i in Duration]
Gross_col_in_million= [float(i.replace('$', '').replace('M', '')) if i != 'N/A' else 'N/A' for i in Gross_col]
Votes = [int(i.replace(',', '')) for i in Votes]

Creating our First CSV file

In [19]:
#Creating dataframe
df=pd.DataFrame({'Sno':sr_number,'Movie_Name': Movie_name,'Director_Name':Directors,'Duration_min':Duration_in_min,'Year':Year,'Ratings':Ratings,'Metascore':Metascore})

In [20]:
df

,Sno,Movie_Name,Director_Name,Duration_min,Year,Ratings,Metascore
0,1,The Dark Knight,Christopher Nolan,152,2008,9.0,84
1,2,The Lord of the Rings: The Return of the King,Peter Jackson,201,2003,9.0,94
2,3,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...",140,2023,8.9,86
3,4,Inception,Christopher Nolan,148,2010,8.8,74
4,5,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,178,2001,8.8,92
...,...,...,...,...,...,...,...
1770,1771,Radhe,Prabhu Deva,113,2021,1.9,N/A
1771,1772,Race 3,Remo D'Souza,160,2018,1.9,N/A
1772,1773,Angels Apocalypse,"Sean Cain, Enzo Zelocchi",85,2015,1.6,N/A
1773,1774,The Cost of Deception,Keith English,125,2021,1.5,N/A


In [21]:
#Bifurcating the Directors name if there are more than one directors of a movie

split_names = df['Director_Name'].str.split(',', expand=True)

max_names = split_names.shape[1]

new_columns = ['Director{}'.format(i) for i in range(1, max_names + 1)]

df[new_columns] = split_names

#Filling N/A if not multiple directoers
df = df.fillna('N/A')

#Dropping Director_name column
df=df.drop(['Director_Name'],axis=1)
df

,Sno,Movie_Name,Duration_min,Year,Ratings,Metascore,Director1,Director2,Director3,Director4,Director5,Director6
0,1,The Dark Knight,152,2008,9.0,84,Christopher Nolan,N/A,N/A,N/A,N/A,N/A
1,2,The Lord of the Rings: The Return of the King,201,2003,9.0,94,Peter Jackson,N/A,N/A,N/A,N/A,N/A
2,3,Spider-Man: Across the Spider-Verse,140,2023,8.9,86,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,N/A,N/A,N/A
3,4,Inception,148,2010,8.8,74,Christopher Nolan,N/A,N/A,N/A,N/A,N/A
4,5,The Lord of the Rings: The Fellowship of the Ring,178,2001,8.8,92,Peter Jackson,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
1770,1771,Radhe,113,2021,1.9,N/A,Prabhu Deva,N/A,N/A,N/A,N/A,N/A
1771,1772,Race 3,160,2018,1.9,N/A,Remo D'Souza,N/A,N/A,N/A,N/A,N/A
1772,1773,Angels Apocalypse,85,2015,1.6,N/A,Sean Cain,Enzo Zelocchi,N/A,N/A,N/A,N/A
1773,1774,The Cost of Deception,125,2021,1.5,N/A,Keith English,N/A,N/A,N/A,N/A,N/A


In [22]:
#Saving the datafrmae to csv file (imdb_data1.csv)
df.to_csv('imdb_data1.csv', index=False)

Creating our Second CSV File

In [23]:
#Creating a dataframe
dfx=pd.DataFrame({'Movie_Name':Movie_name,'Stars':Stars,'Votes':Votes,'Genre':Genre,'Gross_collection_millions':Gross_col_in_million,'Certification':Certification})

In [24]:
dfx

,Movie_Name,Stars,Votes,Genre,Gross_collection_millions,Certification
0,The Dark Knight,"[Christian Bale, Heath Ledger, Aaron Eckhart, ...",2756999,"Action, Crime, Drama",534.86,12A
1,The Lord of the Rings: The Return of the King,"[Elijah Wood, Viggo Mortensen, Ian McKellen, O...",1905424,"Action, Adventure, Drama",377.85,12A
2,Spider-Man: Across the Spider-Verse,"[Shameik Moore, Hailee Steinfeld, Brian Tyree ...",200317,"Animation, Action, Adventure",N/A,PG
3,Inception,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...",2447577,"Action, Adventure, Sci-Fi",292.58,12A
4,The Lord of the Rings: The Fellowship of the Ring,"[Elijah Wood, Ian McKellen, Orlando Bloom, Sea...",1933646,"Action, Adventure, Drama",315.54,PG
...,...,...,...,...,...,...
1770,Radhe,"[Salman Khan, Disha Patani, Randeep Hooda, Jac...",179099,"Action, Crime, Thriller",N/A,15
1771,Race 3,"[Anil Kapoor, Salman Khan, Bobby Deol, Jacquel...",47867,"Action, Crime, Thriller",1.69,12A
1772,Angels Apocalypse,"[Enzo Zelocchi, Jana Rochelle, Ryan C.F. Buckl...",42920,"Action, Fantasy, Sci-Fi",N/A,N/A
1773,The Cost of Deception,"[Vivianne Bánovits, András Mózes, Barna Bokor,...",39616,"Action, Crime, Drama",N/A,N/A


In [25]:
#Bifurcating the Stars name
Stars1=dfx['Stars']
Stars1 = [row for row in Stars1 if row]

# Determine the maximum number of actors in any row
max_actors = max(len(row) for row in Stars1)

# Pad rows with 'N/A' until they all have the same length
for row in Stars1:
    while len(row) < max_actors:
        row.append('N/A')

# Create a DataFrame from the updated list
dfy = pd.DataFrame(Stars1, columns=['Star1', 'Star2', 'Star3', 'Star4'])

In [26]:
dfy

,Star1,Star2,Star3,Star4
0,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
1,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom
2,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez
3,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe
4,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean
...,...,...,...,...
1770,Salman Khan,Disha Patani,Randeep Hooda,Jackie Shroff
1771,Anil Kapoor,Salman Khan,Bobby Deol,Jacqueline Fernandez
1772,Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,William Kirkham
1773,Vivianne Bánovits,András Mózes,Barna Bokor,Gabriella Gubás


In [27]:
dfx[['Star1','Star2','Star3','Star4']]=dfy[['Star1','Star2','Star3','Star4']]

#Dropping Stars column
dfx=dfx.drop(['Stars'],axis=1)

In [28]:
dfx

,Movie_Name,Votes,Genre,Gross_collection_millions,Certification,Star1,Star2,Star3,Star4
0,The Dark Knight,2756999,"Action, Crime, Drama",534.86,12A,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
1,The Lord of the Rings: The Return of the King,1905424,"Action, Adventure, Drama",377.85,12A,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom
2,Spider-Man: Across the Spider-Verse,200317,"Animation, Action, Adventure",N/A,PG,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez
3,Inception,2447577,"Action, Adventure, Sci-Fi",292.58,12A,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe
4,The Lord of the Rings: The Fellowship of the Ring,1933646,"Action, Adventure, Drama",315.54,PG,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean
...,...,...,...,...,...,...,...,...,...
1770,Radhe,179099,"Action, Crime, Thriller",N/A,15,Salman Khan,Disha Patani,Randeep Hooda,Jackie Shroff
1771,Race 3,47867,"Action, Crime, Thriller",1.69,12A,Anil Kapoor,Salman Khan,Bobby Deol,Jacqueline Fernandez
1772,Angels Apocalypse,42920,"Action, Fantasy, Sci-Fi",N/A,N/A,Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,William Kirkham
1773,The Cost of Deception,39616,"Action, Crime, Drama",N/A,N/A,Vivianne Bánovits,András Mózes,Barna Bokor,Gabriella Gubás


In [29]:
#bifurcating the Genre columns into three different columns
dfx[['Genre1', 'Genre2', 'Genre3']] =dfx['Genre'].str.split(', ', expand=True)

#Dropping Genre column
dfx=dfx.drop(['Genre'],axis=1)

In [30]:
dfx

,Movie_Name,Votes,Gross_collection_millions,Certification,Star1,Star2,Star3,Star4,Genre1,Genre2,Genre3
0,The Dark Knight,2756999,534.86,12A,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Action,Crime,Drama
1,The Lord of the Rings: The Return of the King,1905424,377.85,12A,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,Action,Adventure,Drama
2,Spider-Man: Across the Spider-Verse,200317,N/A,PG,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,Animation,Action,Adventure
3,Inception,2447577,292.58,12A,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,Action,Adventure,Sci-Fi
4,The Lord of the Rings: The Fellowship of the Ring,1933646,315.54,PG,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,Action,Adventure,Drama
...,...,...,...,...,...,...,...,...,...,...,...
1770,Radhe,179099,N/A,15,Salman Khan,Disha Patani,Randeep Hooda,Jackie Shroff,Action,Crime,Thriller
1771,Race 3,47867,1.69,12A,Anil Kapoor,Salman Khan,Bobby Deol,Jacqueline Fernandez,Action,Crime,Thriller
1772,Angels Apocalypse,42920,N/A,N/A,Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,William Kirkham,Action,Fantasy,Sci-Fi
1773,The Cost of Deception,39616,N/A,N/A,Vivianne Bánovits,András Mózes,Barna Bokor,Gabriella Gubás,Action,Crime,Drama


In [31]:
#Saving the dataframe into csv file (imbd.data2.csv)
dfx.to_csv('imdb_data2.csv', index=False)

# CSV File 1 =  imdb_data1.csv
# CSV File 2 = imdb_data2.csv